# GEE relative uncertainty retrieval S2 TOA

## Environment setup

In [ ]:
!pip install geemap

In [ ]:
import geemap
import ee

In [ ]:
# Trigger the authentication flow.
ee.Authenticate()
# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=ykAijDUvbkTu0KUhKvwoncgJlgdHr5zSPknctSK24mI&tc=_f4FznZwUniBjL7sLFo_4Oq1Ii-wsyhw08xgXqjxP7U&cc=hRJ91qwABImI4nFmTvhtZZmdGI-03kJgH-6iJfn-PM0

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1Adeu5BXDD52p693YaVjtnwcBup75-pFchKg4wm4ITt0dUN-o6ZzjmHY5oiM

Successfully saved authorization token.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Model import

In [ ]:
# Copy TOA models with uncertainty related hyperparameters contained in TOAModelsUnc.py into the local Colab working directory from Google Drive
# (modify the following path with the Google Drive Path where TOAModelsUnc.py is)
!cp /content/drive/MyDrive/geePythonModels/overall_model_gee_Cm.py .
# Import the models
import overall_model_gee_Cm

In [ ]:
# Possible values: 'Cab', 'Cm', 'Cw', 'FVC', 'LAI', 'laiCab', 'laiCw', 'laiCm'
cropTrait = 'Cm'
uncModel = overall_model_gee_Cm.models[cropTrait]

## Functions

In [ ]:
# Function to mask the clouds in S2
def maskS2cloud_and_water(image):
  not_cloud_shadows = image.select('SCL').neq(3);
  not_water = image.select('SCL').neq(6);
  not_cloud_low = image.select('SCL').neq(7);
  not_cloud_medium = image.select('SCL').neq(8);
  not_cloud_high = image.select('SCL').neq(9);
  not_cirrus = image.select('SCL').neq(10);
  not_ice = image.select('SCL').neq(11);

  qa = image.select('QA60');
  cloudBitMask = 1 << 10;
  cirrusBitMask = 1 << 11;
  mask = (qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0)).And(not_cloud_shadows)
    .And(not_water).And(not_cloud_low).And(not_cloud_medium).And(not_cloud_high).And(not_cirrus).And(not_ice))
  return image.updateMask(mask).divide(10000).copyProperties(qa).set('system:time_start', qa.get('system:time_start'))

# Auxiliar function for mapping : (1..n) -> (B1..Bn)
def band_names(element):
  bandName = ee.String('B').cat(ee.Number(element).int().format())
  return bandName

# Crop trait GPR mean prediction
def veg_index_GPR(image_orig):
  # Create List of Bands of Dimension n (Xtrain[n,n])
  XTrain_dim = uncModel['X_train'].length().get([0]).getInfo();
  band_sequence   = (ee.List.sequence(1, XTrain_dim).map(band_names));
  # Create a list of band names for flattening operation
  im_norm_ell2D_hypell = image_orig.subtract(ee.Image(uncModel['mx'])).divide(ee.Image(uncModel['sx'])).multiply(ee.Image(uncModel['hyp_ell'])).toArray().toArray(1);
  im_norm_ell2D = image_orig.subtract(ee.Image(uncModel['mx'])).divide(ee.Image(uncModel['sx'])).toArray().toArray(1);
  PtTPt  = im_norm_ell2D_hypell.matrixTranspose().matrixMultiply(im_norm_ell2D).arrayProject([0]).multiply(-0.5);
  PtTDX  = ee.Image(uncModel['X_train']).matrixMultiply(im_norm_ell2D_hypell).arrayProject([0]).arrayFlatten([band_sequence]);
  arg1   = PtTPt.exp().multiply(uncModel['hyp_sig']);
  k_star = PtTDX.subtract(ee.Image(uncModel['XDX_pre_calc']).multiply(0.5)).exp().toArray();
  mean_pred = k_star.arrayDotProduct(ee.Image(uncModel['alpha_coefficients']).toArray()).multiply(arg1);
  mean_pred = mean_pred.toArray(1).arrayProject([0]).arrayFlatten([[uncModel['veg_index']]]);
  mean_pred = mean_pred.add(uncModel['mean_model']);
  image_orig = image_orig.addBands(mean_pred)
  return image_orig.select(uncModel['veg_index'])

# Uncertainty retrieval
def get_GPR_uncertainty(image_orig):
  # Create List of Bands of Dimension n (Xtrain[n,n])
  XTrain_dim = uncModel['X_train'].length().get([0]).getInfo()
  band_sequence   = (ee.List.sequence(1, XTrain_dim).map(band_names))
  # Create a list of band names for flattening operation
  im_norm_ell2D_hypell = image_orig.subtract(ee.Image(uncModel['mx'])).divide(ee.Image(uncModel['sx'])).multiply(ee.Image(uncModel['hyp_ell'])).toArray().toArray(1)
  im_norm_ell2D = image_orig.subtract(ee.Image(uncModel['mx'])).divide(ee.Image(uncModel['sx'])).toArray().toArray(1)
  PtTPt  = im_norm_ell2D_hypell.matrixTranspose().matrixMultiply(im_norm_ell2D).arrayProject([0]).multiply(-0.5)
  PtTDX  = ee.Image(uncModel['X_train']).matrixMultiply(im_norm_ell2D_hypell).arrayProject([0]).arrayFlatten([band_sequence])
  arg1   = PtTPt.exp().multiply(uncModel['hyp_sig']);
  k_star = PtTDX.subtract(ee.Image(uncModel['XDX_pre_calc']).multiply(0.5)).exp().multiply(arg1).toArray()

  variance_vector = ee.Image(uncModel['Linv_pre_calc']).matrixMultiply(k_star.toArray(0).toArray(1)).arrayProject([0])
  variance = ee.Image(uncModel['hyp_sig_unc']).toArray().subtract(variance_vector.arrayDotProduct(variance_vector))
  variance = variance.toArray(1).arrayProject([0]).arrayFlatten([[uncModel['veg_index']+'_unc']])
  image_orig = image_orig.addBands(variance)
  return image_orig.select(uncModel['veg_index']+'_unc').abs().sqrt()

## Area of interest parameters

In [ ]:
polygon = ee.Geometry.Polygon(
  [[[104.94710610605645,8.605965246171419],
    [104.94710610605645,8.669099010946878],
    [104.76445840097833,8.669099010946878],
    [104.76445840097833,8.605965246171419]]])

studyArea = ee.FeatureCollection('projects/ee-ngabinh1987/assets/DiaPhan_VQG_DatMui')

## Mangrove Vegetation Index (MVI)

In [ ]:
annual_collection  = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
  .filterBounds(polygon)
  .filterDate('2019-01-01', '2022-12-31')
  .map(maskS2cloud_and_water)
  .select(['B3', 'B8', 'B11']))

mean_annual_image = annual_collection.median().clip(studyArea);

MVI = mean_annual_image.expression('(b("B8") - b("B3"))/(b("B11") - b("B3"))');

def classify_mangrove(img):
  band = img.select('B8')
  mangrove = band.gt(4.5).rename('mangrove')
  mangrove = mangrove.updateMask(mangrove)
  return img.addBands(mangrove)

MVI_classify = classify_mangrove(MVI);

def clip_StudyArea(image):return image.clip(studyArea)
def clip_MVI(image):return image.updateMask(MVI_classify.select('mangrove'))

## Traits, Uncertainty, Relative Uncertainty

In [ ]:
# Image Collection
img_col = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
  .filterBounds(polygon)
  .filterDate('2023-06-01', '2023-06-30')
  .map(maskS2cloud_and_water)
  .select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B11', 'B12'])
  .map(clip_StudyArea)
  .map(clip_MVI));

# Predicted mean trait
trait = img_col.map(veg_index_GPR).mean().copyProperties(
  source = img_col.first(),
  properties = ['system:time_start']
  );

# Absolute uncertainty retrieval (SD)
SD = img_col.map(get_GPR_uncertainty).mean().copyProperties(
  source = img_col.first(),
  properties = ['system:time_start']
  );

# Convert to image (mặc dù các kết quả tính ra đã là image nhưng nếu ko covert lại một lần nữa vẫn bị lỗi)
imageTrait = ee.Image(trait)
imageSD = ee.Image(SD)

# Relative uncertainty retrieval (CV)
imageCV = imageSD.divide(imageTrait).multiply(100).rename([cropTrait+'_CV'])

imageSD

## Image export as an asset.

In [ ]:
# In order to visualize the image in this notebook avoiding memory problems,
# the image has to be exported to the asset section.

# GEE user name
geeUser = 'ngabinh1987/Cm_unc_monthly_each_year'
# Asset name
assetName = 'Cm_SD_2023_06'

assetId = 'users/' + geeUser + '/' + assetName

task_config = {
    'image': imageSD,
    'description': 'Cm_SD_2023_06',
    'assetId': assetId,
    'region': studyArea.geometry(),
    'scale': 10,
    'crs' : 'EPSG:4326'
}

task = ee.batch.Export.image.toAsset(**task_config)
task.start()

### Check the status of the export task

In [ ]:
# To visualize the image, the status must be "COMPLETED"
desc = task.status()['description']
state = task.status()['state']

print('The status of the task {} is: {}'.format(desc,state))

The status of the task Cab_SD_2020_05 is: READY


## Visualization

### Visualization parameters

In [ ]:
CV_palette = ['000083','000087','00008B','00008F','000093','000097','00009B','00009F','0000A3','0000A7','0000AB','0000AF','0000B3','0000B7','0000BB','0000BF','0000C3',
              '0000C7','0000CB','0000CF','0000D3','0000D7','0000DB','0000DF','0000E3','0000E7','0000EB','0000EF','0000F3','0000F7','0000FB','0000FF','0003FF','0007FF',
              '000BFF','000FFF','0013FF','0017FF','001BFF','001FFF','0023FF','0027FF','002BFF','002FFF','0033FF','0037FF','003BFF','003FFF','0043FF','0047FF','004BFF',
              '004FFF','0053FF','0057FF','005BFF','005FFF','0063FF','0067FF','006BFF','006FFF','0073FF','0077FF','007BFF','007FFF','0083FF','0087FF','008BFF','008FFF',
              '0093FF','0097FF','009BFF','009FFF','00A3FF','00A7FF','00ABFF','00AFFF','00B3FF','00B7FF','00BBFF','00BFFF','00C3FF','00C7FF','00CBFF','00CFFF','00D3FF',
              '00D7FF','00DBFF','00DFFF','00E3FF','00E7FF','00EBFF','00EFFF','00F3FF','00F7FF','00FBFF','00FFFF','03FFFB','07FFF7','0BFFF3','0FFFEF','13FFEB','17FFE7',
              '1BFFE3','1FFFDF','23FFDB','27FFD7','2BFFD3','2FFFCF','33FFCB','37FFC7','3BFFC3','3FFFBF','43FFBB','47FFB7','4BFFB3','4FFFAF','53FFAB','57FFA7','5BFFA3',
              '5FFF9F','63FF9B','67FF97','6BFF93','6FFF8F','73FF8B','77FF87','7BFF83','7FFF7F','83FF7B','87FF77','8BFF73','8FFF6F','93FF6B','97FF67','9BFF63','9FFF5F',
              'A3FF5B','A7FF57','ABFF53','AFFF4F','B3FF4B','B7FF47','BBFF43','BFFF3F','C3FF3B','C7FF37','CBFF33','CFFF2F','D3FF2B','D7FF27','DBFF23','DFFF1F','E3FF1B',
              'E7FF17','EBFF13','EFFF0F','F3FF0B','F7FF07','FBFF03','FFFF00','FFFB00','FFF700','FFF300','FFEF00','FFEB00','FFE700','FFE300','FFDF00','FFDB00','FFD700',
              'FFD300','FFCF00','FFCB00','FFC700','FFC300','FFBF00','FFBB00','FFB700','FFB300','FFAF00','FFAB00','FFA700','FFA300','FF9F00','FF9B00','FF9700','FF9300',
              'FF8F00','FF8B00','FF8700','FF8300','FF7F00','FF7B00','FF7700','FF7300','FF6F00','FF6B00','FF6700','FF6300','FF5F00','FF5B00','FF5700','FF5300','FF4F00',
              'FF4B00','FF4700','FF4300','FF3F00','FF3B00','FF3700','FF3300','FF2F00','FF2B00','FF2700','FF2300','FF1F00','FF1B00','FF1700','FF1300','FF0F00','FF0B00',
              'FF0700','FF0300','FF0000','FB0000','F70000','F30000','EF0000','EB0000','E70000','E30000','DF0000','DB0000','D70000','D30000','CF0000','CB0000','C70000',
              'C30000','BF0000','BB0000','B70000','B30000','AF0000','AB0000','A70000','A30000','9F0000','9B0000','970000','930000','8F0000','8B0000','870000','830000',
              '7F0000']

vis_params = {
    'min': 0.0,
    'max': 100.0,
    'palette': CV_palette
}

### Visualization through geemap

In [ ]:
image = ee.Image(currentImage)

Map = geemap.Map()
Map.add_basemap('SATELLITE')
Map.addLayer(image, vis_params, 'CVV uncertainty')
Map.add_colorbar(vis_params, label = 'CVV', layer_name = 'CVV', position = 'topright')
Map.centerObject(studyArea)
Map

NameError: ignored

## References

Wu, Q., (2020). geemap: A Python package for interactive mapping with Google Earth Engine. The Journal of Open Source Software, 5(51), 2305. https://doi.org/10.21105/joss.02305